## Setup

Import TensorFlow 2.0:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt

## Exercise on in-graph training loop

This exercise teaches how to train a Keras model on horses or humans dataset with the entire training process—loading batches, calculating gradients, updating parameters, calculating validation accuracy, and repeating until convergence—is performed in-graph.

### Prepare the dataset

In [ ]:
splits = tfds.Split.ALL.subsplit(weighted=(80, 10, 10))
splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=splits)

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

In [ ]:
@tf.function
def map_fn(img, label):
  img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
  img /= 255.0
  return img, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_examples.map(map_fn).shuffle(buffer_size=num_examples).batch(BATCH_SIZE)
valid_ds = validation_examples.map(map_fn).batch(BATCH_SIZE)
test_ds = test_examples.map(map_fn).batch(BATCH_SIZE)

### Define the model

In [ ]:
MODULE_HANDLE = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

## Deine optimizer, loss and metrics

In [ ]:
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.losses.SparseCategoricalCrossentropy()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
val_loss = tf.keras.losses.SparseCategoricalCrossentropy()
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

### Define the training loop

In [ ]:
device = '/gpu:0' if tf.test.is_gpu_available() else '/cpu:0'

In [ ]:
EPOCHS = 2

In [ ]:
# Custom training step
def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    predictions = model(x)
    loss = train_loss(y, predictions)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_accuracy(y, predictions)
  return loss

@tf.function
def train(model, optimizer):
  step = 0
  loss = 0.0
  for epoch in range(EPOCHS):
    for x, y in train_ds:
      step += 1
      with tf.device(device_name=device):
        loss = train_one_step(model, optimizer, x, y)
      tf.print('Step', step, 
               ': train loss', loss, 
               '; train accuracy', train_accuracy.result())

    with tf.device(device_name=device):
      for x, y in valid_ds:
        y_pred = model(x)
        loss = val_loss(y, y_pred)
        val_accuracy(y, y_pred)

    tf.print('val loss', loss, '; val accuracy', val_accuracy.result())

In [ ]:
train(model, optimizer)

# Evaluation

In [ ]:
test_imgs = []
test_labels = []

predictions = []
with tf.device(device_name=device):
  for images, labels in test_ds:
    preds = model(images)
    preds = preds.numpy()
    predictions.extend(preds)
    
    test_imgs.extend(images.numpy())
    test_labels.extend(labels.numpy())

In [ ]:
#@title Utility functions for plotting
# Utilities for plotting

class_names = ['horse', 'human']

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
    
  img = np.squeeze(img)

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'
  print(true_label)
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)



In [ ]:
#@title Visualize the outputs { run: "auto" }
index = 8 #@param {type:"slider", min:0, max:9, step:1}
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(index, predictions, test_labels, test_imgs)
plt.show()